# Imputation Research Project <img src="https://chroniclesofai.com/content/images/2021/05/file-20201210-18-elk4m.jpg" alt="Alt text image not displaying" width="360" align="right" />
## Notebook 3.0: Autoencoder Model

**Author:** Chike Odenigbo

**Date:** November 25th, 2022

**Notebook Structure:**

* 1.0 Preprocessing

* **1.1 Exploratory Data Analysis**

* 1.2 Masking

* 2.* Models


Water Sugar lutein_zeaxanthin Alcohol

In [2]:
import pandas as pd
import os
from src.visualization.visualize import histogram, box_plot, bar_plot
from pathlib import Path
from notebook_config import ROOT_DIR  # setup.py file changed the root of the project so it is set in the config file

ROOT_DIR = ROOT_DIR.as_posix()  # convert root path to windows readable path (i.e. change backslash to forward slash)
from joblib import load
import tensorflow as tf
from keras import backend as K
import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import pickle

In [38]:
notebook_nm = "3.0-autoencoder"
fig_dir = f"{ROOT_DIR}/reports/figures/"
model_dir = f"{ROOT_DIR}/models/autoencoders/"
scaler_dir = f'{ROOT_DIR}/models/scalers/'
output_prefix = notebook_nm

In [37]:
pd.set_option('display.max_columns', None)

# Ground Truth Included
water_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water.csv')
sugars_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars.csv')
lutein_df = pd.read_csv(f'{ROOT_DIR}/data/processed/lutein.csv')

# Scaled Data without Ground Truth to prevent Data Leakage, rows are included with NaN
water_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mcar_scaled.csv', index_col = 0)
water_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mar_scaled.csv', index_col = 0)

lutein_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/lutein_mar_scaled.csv', index_col = 0)
lutein_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/utein_mcar_scaled.csv', index_col = 0)

sugars_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars_mar_scaled.csv', index_col = 0)
sugars_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars_mcar_scaled.csv', index_col = 0)

water_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mcar_scaled.csv', index_col = 0)
water_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mar_scaled.csv', index_col = 0)

# Scalers to return back to origin scale for model evaluation
scaler_lutein_mar = load(f'{scaler_dir}/scaler_lutein_mar.joblib')
scaler_lutein_mcar = load(f'{scaler_dir}/scaler_lutein_mcar.joblib')

scaler_sugars_mar = load(f'{scaler_dir}/scaler_sugars_mar.joblib')
scaler_sugars_mcar = load(f'{scaler_dir}/scaler_sugars_mcar.joblib')

scaler_water_mcar = load(f'{scaler_dir}/scaler_water_mcar.joblib')
scaler_water_mar = load(f'{scaler_dir}/scaler_water_mar.joblib')

In [52]:
def scale_ground_truth(scaler,raw_df,target_col,non_target_na_col):
    raw_filtered_df = raw_df[raw_df[target_col].isnull()].drop(['name',target_col, non_target_na_col], axis = 'columns')
    scaled_ground_truth_df = pd.DataFrame(scaler.transform(raw_filtered_df),columns = raw_filtered_df.columns)
    return scaled_ground_truth_df

def train_test_split_scaled(scaled_df,drop_cols = ['name','dataset_type','serving_size']):
    train_df = scaled_df[scaled_df.dataset_type == 'training'].drop(drop_cols, axis = 'columns')
    val_df = scaled_df[scaled_df.dataset_type == 'validation'].drop(drop_cols, axis = 'columns')
    return train_df, val_df

def autoencoder(data, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.0001):
    enc_input = keras.Input(shape = (input_shape,),name = 'input_data')
    latent_layer = keras.layers.Dense(latent_shape, activation = 'relu')(enc_input)
    encoder = keras.Model(enc_input,latent_layer, name = "encoder")
    dec_output = keras.layers.Dense(input_shape)(latent_layer)
    opt = keras.optimizers.Adam(lr = lr)
    auto_encoder = keras.Model(enc_input,dec_output, name="auto_encoder")
    print(auto_encoder.summary())
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    auto_encoder.compile(opt, loss = "mse")
    history = auto_encoder.fit(data, data, epochs = 1000, validation_split = 0.1, callbacks=[callback])
    return auto_encoder, history

def save_autoencoder(model,model_name,path = model_dir):
    model.save(f'{path}/{model_name}.h5')
    with open(f'{path}/{model_name}_history.pkl', 'wb') as file_pi:
        pickle.dump(model.history, file_pi)

In [14]:
water_mcar_ground_truth_scaled_df = scale_ground_truth(scaler = scaler_water_mcar,raw_df = water_df,target_col = 'water_mcar',non_target_na_col = 'water_mar')
water_mar_ground_truth_scaled_df = scale_ground_truth(scaler = scaler_water_mar,raw_df = water_df,target_col = 'water_mar',non_target_na_col = 'water_mcar')

sugars_mcar_ground_truth_scaled_df = scale_ground_truth(scaler = scaler_sugars_mcar,raw_df = sugars_df,target_col = 'sugars_mcar',non_target_na_col = 'sugars_mar')
sugars_mar_ground_truth_scaled_df = scale_ground_truth(scaler = scaler_sugars_mar,raw_df = sugars_df,target_col = 'sugars_mar',non_target_na_col = 'sugars_mcar')

lutein_mcar_ground_truth_scaled_df = scale_ground_truth(scaler = scaler_lutein_mcar,raw_df = lutein_df,target_col = 'lutein_zeaxanthin_mcar',non_target_na_col = 'lutein_zeaxanthin_mar')
lutein_mar_ground_truth_scaled_df = scale_ground_truth(scaler = scaler_lutein_mar,raw_df = lutein_df,target_col = 'lutein_zeaxanthin_mar',non_target_na_col = 'lutein_zeaxanthin_mcar')


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- water
Feature names seen at fit time, yet now missing:
- water_mcar

  warnings.warn(message, FutureWarning)
C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- water
Feature names seen at fit time, yet now missing:
- water_mar

  warnings.warn(message, FutureWarning)
C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- sugars
Feature names see

In [25]:
water_mcar_train_df, water_mcar_val_df = train_test_split_scaled(water_mcar_scaled_df,drop_cols = ['name','dataset_type','serving_size'])
water_mar_train_df, water_mar_val_df = train_test_split_scaled(water_mar_scaled_df,drop_cols = ['name','dataset_type','serving_size'])

sugars_mcar_train_df, sugars_mcar_val_df = train_test_split_scaled(sugars_mcar_scaled_df,drop_cols = ['name','dataset_type','serving_size'])
sugars_mar_train_df, sugars_mar_val_df = train_test_split_scaled(sugars_mar_scaled_df,drop_cols = ['name','dataset_type','serving_size'])

lutein_mcar_train_df, lutein_mcar_val_df = train_test_split_scaled(lutein_mcar_scaled_df,drop_cols = ['name','dataset_type','serving_size'])
lutein_mar_train_df, lutein_mar_val_df = train_test_split_scaled(lutein_mar_scaled_df,drop_cols = ['name','dataset_type','serving_size'])


In [30]:
sugars_mcar_model, sugars_mcar_history = autoencoder(sugars_mcar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_4 (Dense)             (None, 40)                2920      
                                                                 
 dense_5 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


140/140 [==============================] - 1s 4ms/step - loss: 0.8909 - val_loss: 0.1615
Epoch 2/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.5482 - val_loss: 0.1129
Epoch 3/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.4145 - val_loss: 0.0921
Epoch 4/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.3333 - val_loss: 0.0795
Epoch 5/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.2784 - val_loss: 0.0700
Epoch 6/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.2375 - val_loss: 0.0632
Epoch 7/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.2067 - val_loss: 0.0542
Epoch 8/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.1816 - val_loss: 0.0503
Epoch 9/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.1616 - val_loss: 0.0473
Epoch 10/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.1437 - val_loss:

In [31]:
sugars_mar_model, sugars_mar_history = autoencoder(sugars_mar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_6 (Dense)             (None, 40)                2920      
                                                                 
 dense_7 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


None
Epoch 1/1000
183/183 [==============================] - 1s 3ms/step - loss: 0.5773 - val_loss: 0.1097
Epoch 2/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.3271 - val_loss: 0.0787
Epoch 3/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.2425 - val_loss: 0.0609
Epoch 4/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1885 - val_loss: 0.0511
Epoch 5/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1530 - val_loss: 0.0443
Epoch 6/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1278 - val_loss: 0.0423
Epoch 7/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1090 - val_loss: 0.0391
Epoch 8/1000
183/183 [==============================] - 0s 3ms/step - loss: 0.0947 - val_loss: 0.0352
Epoch 9/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.0831 - val_loss: 0.0338
Epoch 10/1000
183/183 [==============================] - 0s 2ms/step - loss: 

In [32]:
water_mar_model, water_mar_history = autoencoder(water_mar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_8 (Dense)             (None, 40)                2920      
                                                                 
 dense_9 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


None
Epoch 1/1000
183/183 [==============================] - 1s 3ms/step - loss: 0.5975 - val_loss: 0.1203
Epoch 2/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.3369 - val_loss: 0.0904
Epoch 3/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.2500 - val_loss: 0.0700
Epoch 4/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1947 - val_loss: 0.0536
Epoch 5/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1575 - val_loss: 0.0459
Epoch 6/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1305 - val_loss: 0.0380
Epoch 7/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.1101 - val_loss: 0.0352
Epoch 8/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.0954 - val_loss: 0.0325
Epoch 9/1000
183/183 [==============================] - 0s 2ms/step - loss: 0.0840 - val_loss: 0.0302
Epoch 10/1000
183/183 [==============================] - 0s 3ms/step - loss: 

In [33]:
water_mcar_model, water_mcar_history = autoencoder(water_mcar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_10 (Dense)            (None, 40)                2920      
                                                                 
 dense_11 (Dense)            (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


161/161 [==============================] - 1s 3ms/step - loss: 0.8786 - val_loss: 0.2011
Epoch 2/1000
161/161 [==============================] - 0s 2ms/step - loss: 0.5347 - val_loss: 0.1472
Epoch 3/1000
161/161 [==============================] - 0s 3ms/step - loss: 0.4113 - val_loss: 0.1211
Epoch 4/1000
161/161 [==============================] - 0s 2ms/step - loss: 0.3307 - val_loss: 0.1060
Epoch 5/1000
161/161 [==============================] - 0s 3ms/step - loss: 0.2711 - val_loss: 0.0953
Epoch 6/1000
161/161 [==============================] - 0s 3ms/step - loss: 0.2282 - val_loss: 0.0877
Epoch 7/1000
161/161 [==============================] - 0s 3ms/step - loss: 0.1949 - val_loss: 0.0823
Epoch 8/1000
161/161 [==============================] - 0s 2ms/step - loss: 0.1701 - val_loss: 0.0752
Epoch 9/1000
161/161 [==============================] - 0s 2ms/step - loss: 0.1510 - val_loss: 0.0742
Epoch 10/1000
161/161 [==============================] - 0s 3ms/step - loss: 0.1355 - val_loss:

In [34]:
lutein_mar_model, lutein_mar_history = autoencoder(lutein_mar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_12 (Dense)            (None, 40)                2920      
                                                                 
 dense_13 (Dense)            (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


None
Epoch 1/1000
162/162 [==============================] - 1s 3ms/step - loss: 0.6495 - val_loss: 0.1221
Epoch 2/1000
162/162 [==============================] - 0s 3ms/step - loss: 0.3735 - val_loss: 0.0919
Epoch 3/1000
162/162 [==============================] - 0s 3ms/step - loss: 0.2765 - val_loss: 0.0735
Epoch 4/1000
162/162 [==============================] - 0s 2ms/step - loss: 0.2194 - val_loss: 0.0604
Epoch 5/1000
162/162 [==============================] - 0s 2ms/step - loss: 0.1794 - val_loss: 0.0511
Epoch 6/1000
162/162 [==============================] - 0s 2ms/step - loss: 0.1502 - val_loss: 0.0437
Epoch 7/1000
162/162 [==============================] - 0s 2ms/step - loss: 0.1277 - val_loss: 0.0389
Epoch 8/1000
162/162 [==============================] - 0s 2ms/step - loss: 0.1104 - val_loss: 0.0356
Epoch 9/1000
162/162 [==============================] - 0s 2ms/step - loss: 0.0973 - val_loss: 0.0338
Epoch 10/1000
162/162 [==============================] - 0s 3ms/step - loss: 

In [35]:
lutein_mcar_model, lutein_mcar_history = autoencoder(lutein_mcar_train_df, input_shape = 72, latent_shape = 40, patience = 2, lr = 0.001)

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_14 (Dense)            (None, 40)                2920      
                                                                 
 dense_15 (Dense)            (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


140/140 [==============================] - 1s 4ms/step - loss: 0.9640 - val_loss: 0.1636
Epoch 2/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.5907 - val_loss: 0.1110
Epoch 3/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.4545 - val_loss: 0.0912
Epoch 4/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.3628 - val_loss: 0.0772
Epoch 5/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.2994 - val_loss: 0.0670
Epoch 6/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.2542 - val_loss: 0.0592
Epoch 7/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.2196 - val_loss: 0.0548
Epoch 8/1000
140/140 [==============================] - 0s 2ms/step - loss: 0.1916 - val_loss: 0.0501
Epoch 9/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.1680 - val_loss: 0.0484
Epoch 10/1000
140/140 [==============================] - 0s 3ms/step - loss: 0.1482 - val_loss:

In [47]:
import json
import pickle
lutein_mcar_model.save(f'{model_dir}/lutein_mcar_model.h5')
with open(f'{model_dir}/lutein_mcar_model.pkl', 'wb') as file_pi:
    pickle.dump(lutein_mcar_model.history, file_pi)
# json.dump(lutein_mcar_model.history, open(f'{model_dir}/lutein_mcar_model.json', 'w'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:26:47         1884
metadata.json                                  2022-11-28 14:26:47           64
variables.h5                                   2022-11-28 14:26:47        36504


In [60]:
save_autoencoder(model = sugars_mcar_model, model_name = 'sugars_mcar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:43:02         1878
metadata.json                                  2022-11-28 14:43:02           64
variables.h5                                   2022-11-28 14:43:02        36504


In [61]:
save_autoencoder(model = sugars_mar_model, model_name = 'sugars_mar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:43:14         1878
metadata.json                                  2022-11-28 14:43:14           64
variables.h5                                   2022-11-28 14:43:14        36504


In [62]:
save_autoencoder(model = water_mar_model, model_name = 'water_mar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:43:19         1878
metadata.json                                  2022-11-28 14:43:19           64
variables.h5                                   2022-11-28 14:43:19        36504


In [63]:
save_autoencoder(model = water_mcar_model, model_name = 'water_mcar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:43:25         1884
metadata.json                                  2022-11-28 14:43:25           64
variables.h5                                   2022-11-28 14:43:25        36504


In [64]:
save_autoencoder(model = lutein_mar_model, model_name = 'lutein_mar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:43:30         1884
metadata.json                                  2022-11-28 14:43:30           64
variables.h5                                   2022-11-28 14:43:30        36504


In [65]:
save_autoencoder(model = lutein_mcar_model, model_name = 'lutein_mcar_model',path = model_dir)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:43:34         1884
metadata.json                                  2022-11-28 14:43:34           64
variables.h5                                   2022-11-28 14:43:34        36504


In [50]:
pd.read_pickle(f'{model_dir}/lutein_mcar_model.pkl').history['val_loss']

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-11-28 14:26:46         1884
metadata.json                                  2022-11-28 14:26:46           64
variables.h5                                   2022-11-28 14:26:46        36504
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...vars


[0.16361281275749207,
 0.11096788197755814,
 0.09116950631141663,
 0.07717607170343399,
 0.06695730239152908,
 0.059187181293964386,
 0.05479991063475609,
 0.050138313323259354,
 0.04843691363930702,
 0.04356687515974045,
 0.04156889021396637,
 0.040680475533008575,
 0.037525858730077744,
 0.037696462124586105,
 0.0358843095600605,
 0.03356218338012695,
 0.03332123905420303,
 0.032371897250413895,
 0.028929756954312325,
 0.028011348098516464,
 0.02625330723822117,
 0.024603238329291344,
 0.02331257425248623,
 0.022573130205273628,
 0.02091863565146923,
 0.020417097955942154,
 0.01877906173467636,
 0.018627813085913658,
 0.01766442321240902,
 0.01716536097228527,
 0.015905698761343956,
 0.015277484431862831,
 0.01410331018269062,
 0.014139523729681969,
 0.01332293264567852,
 0.013186094351112843,
 0.012723461724817753,
 0.014205187559127808,
 0.012737993150949478]

In [42]:
test = keras.models.load_model(f'{model_dir}/lutein_mcar_model.h5')
test.history

In [34]:
max_water_mcar = water_mcar_train_df.water_mcar.max()
min_water_mcar = water_mcar_train_df.water_mcar.min()

In [67]:
water_mcar_test_df['water_mcar'] = 0#np.random.uniform(min_water_mcar, max_water_mcar, size=len(water_mcar_test_df))

In [37]:
water_mcar_test_df

,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water_mcar
1,-0.619342,-0.551556,-0.366113,-0.419401,-0.264566,-0.377104,-0.198124,-0.121463,0.295053,0.016684,0.315431,-0.165265,-0.119859,-0.06443,-0.101149,-0.039304,-0.039796,-0.312098,0.387865,-0.120460,-0.125774,-0.224649,-0.224649,-0.106650,0.565155,1.057292,0.776284,2.507023,1.367768,1.442629,0.469791,-0.355989,0.378058,0.005471,0.247723,-0.195034,-0.083019,0.785573,0.862210,-0.076787,-0.062212,-0.367017,0.059323,0.257674,-0.452719,0.554252,0.543382,0.213058,0.379865,0.157415,0.193964,0.252622,0.283687,1.776759,1.304084,-0.352894,0.311358,-0.062457,0.372412,-0.07869,-0.150615,0.023213,-0.620268,-0.544198,-0.549419,-0.273112,-0.366113,-0.024432,0.265352,-0.032626,-0.083373,-0.003348
2,-0.643570,-0.435218,-0.358304,-0.373641,-0.361717,-0.346635,-0.198124,-0.838286,-0.244720,-0.411286,-0.437932,-0.154655,-0.105630,-0.06443,-0.105163,0.004479,-0.109427,-0.283870,-0.564302,-0.070570,-0.125774,-0.241400,-0.241400,-0.137364,-0.136664,-0.267279,-0.475648,-0.467465,-0.075670,-0.778567,-0.580093,-0.455583,-0.510558,-1.183777,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.779530,-0.854175,-0.367017,-0.891171,-0.904661,-0.834197,-0.856819,-0.915290,-0.882456,-0.907343,-0.882938,-0.868764,-0.882973,-0.905697,0.257549,-0.189534,1.322267,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.643289,-0.440772,-0.557515,-0.454119,-0.358304,-0.024432,-0.642443,-0.032626,-0.083373,1.408102
3,-0.746538,-0.595182,-0.366113,-0.395175,0.296750,0.057075,-0.198124,-0.741841,0.088316,-0.502818,-0.390198,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.116508,-0.312098,-0.230317,0.925066,-0.125774,-0.224649,-0.224649,0.113204,-0.314904,-0.248647,-0.428847,-0.349161,-0.053148,-0.755727,0.063794,-0.486491,-0.569799,-1.103844,-0.687716,-0.732221,-0.677402,-0.742383,-0.796040,-0.674929,-0.706833,-0.367017,-0.756470,-0.789298,-0.614036,-0.790881,-0.779454,-0.765315,-0.729369,-0.727905,-0.715853,-0.756520,-0.688973,-0.646582,-0.033484,-0.347678,0.814953,-0.062457,0.520483,-0.07869,-0.173220,-0.156322,-0.747491,-0.590602,-0.625578,-0.463069,-0.366113,-0.024432,-0.476551,-0.032626,-0.083373,1.651227
13,-0.710196,-0.580640,-0.366113,-0.388446,-0.500247,-0.232377,-0.198124,-0.495732,-0.325912,-0.282648,0.292602,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.456431,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.270344,2.032935,-0.164755,-0.197057,0.005816,-0.750017,0.974116,-0.483057,-0.253846,-0.160244,-0.271142,-0.325903,0.762798,0.509692,-0.504827,-0.204961,-0.220078,-0.367017,-0.084863,-0.208130,-0.233572,-0.385363,0.027205,0.237806,0.052889,0.037060,1.057909,-0.007722,0.132847,0.175776,-0.479340,-0.490007,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.709930,-0.577219,-0.598133,-0.436950,-0.366113,-0.024432,0.094852,-0.032626,-0.083373,-1.192960
20,-0.740481,-0.595182,-0.366113,-0.426130,-0.500247,-0.377104,-0.198124,-0.851971,-0.413117,-0.651247,-0.493967,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.612014,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.326044,-0.314706,-0.499049,-0.602669,-0.077391,-1.006968,-0.884591,-0.489926,-0.645966,-1.291004,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.

In [69]:
water_mcar_pred = pd.DataFrame(auto_encoder.predict(water_mcar_test_df), columns = water_mcar_test_df.columns)

47/47 [==============================] - 0s 1ms/step


In [71]:
water_mcar_true = pd.DataFrame(scaler_water_mcar.transform(water_df[water_df.water_mcar.isnull()].drop(['name','water_mcar', 'water_mar'], axis = 'columns')),columns = water_df.drop(['name','water_mcar', 'water_mar'], axis = 'columns').columns).drop('serving_size',axis = 'columns')# a = np.array(x) # your x
# b = np.array(y) # your y
# mses = ((a-b)**2).mean(axis=1)

C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- water
Feature names seen at fit time, yet now missing:
- water_mcar

  warnings.warn(message, FutureWarning)


In [77]:
from sklearn.metrics import mean_squared_error
pd.DataFrame({'columns':water_mcar_true.columns,
              'mse':list(mean_squared_error(water_mcar_true, water_mcar_pred, multioutput='raw_values'))}).nlargest(10,'mse')

,columns,mse
31,selenium,1.593192
57,galactose,0.668080
58,glucose,0.558205
29,phosphorous,0.404909
6,folic_acid,0.256179
56,fructose,0.250337
71,water,0.208576
5,folate,0.201098
33,protein,0.189536
17,vitamin_b12,0.161144
